Analyzing the NYC Subway Dataset
=================

Secction 0. References
---------------------

* [Welch's t test](http://en.wikipedia.org/wiki/Welch%27s_t_test)
* [Mann-Whitney U test](http://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test)

Section 1. Statistical Test
---------------------------

### 1.1

I used Mann-Whitney U Test to analyze the NYC subway data.

I used a two-tail P value.

The null hypothesis is the subway riders are not affected by whether it rains or not, i.e. the number of riders on rainy days and number of riders on non-rainy days follow the same distribution.

The P-critical value I use is 5%.

### 1.2

The Mann-Whitney U Test is applicable to the dataset is a nonparametric test that does not make any assumption on the underlying distribution. By ploting the histogram of two ridership samples, we see they are clearly not normal distribution, and therefore a t test is not suitable for this dataset.

### 1.3

The results we get from this statistical test is that the ridership **is** affected by whether it rains or not, in other words, the number of riders on rainy days and number of riders on non-rainy days **do not** follow the same distribution. Furthermore, the number of riders on a rainy day is slightly larger than that on a non-rainy day.

The p-value of this test is 0.025. The mean number of riders on rainy days is 1105, and that on non-rainy days is 1090.

### 1.4

The significance level of this test is 5% (the P-critical value I set in 1.1). The interpretation is that the probability of our conclusion is wrong because of bad luck is less than 5%. More specifically, if the ridership is not affected by raining or not, the probability of seeing data as extreme as the one we currently have is less than 5%.

Section 2. Linear Regression
---------------------------

Section 3. Visualization
-----------------------

Section 4. Conclusion
--------------------

Section 5. Reflection
--------------------